### BERT Topic Modelling 
https://maartengr.github.io/BERTopic/index.html

Some support functions to prepare text. BERTopic accepts list of str [str] as input. Cleaning is not mandatory. Normally use unckleaned text. But in the test case I used [sentences] and to get sentence tokens you need to clean 

In [ ]:
import re
import pandas as pd
from nltk.tokenize import sent_tokenize
#################################################################################
def clean_text(text,punctuation):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    if punctuation:
        text = re.sub('\[.*?!\]', ' ', text)#remove !
        text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)#remove other punctuations
    else:
        text = text.lower() #to lower
        #text = re.sub('\w*\d\w*', '', text) #clearing digits
        '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
        text = re.sub('[\n*]', ' . ', text)
        text = re.sub('[/‘’“”…]', '', text)
        text = re.sub('\u20AC', '', text)#remove € 
        text = re.sub('\xa0', ' ', text)
        text=multi_del_recurse(text,'\.')
    return text# indexing the words in sentences with running index numbers and returns a list of dicts or json

#################################################################################
'#recursisive deletion of multi characters ch1'
def multi_del_recurse(t,ch1):
    ch=ch1+ch1
    if re.search(ch, t)!= None :
        t1=re.sub(r'\\','','\.')
        t=re.sub(ch,t1,t)
        return(multi_del_recurse(t,ch1))
    else:
        return t
#################################################################################    
def load_files():
    
    jd_txt=open('Z:\\SEMP_JD.txt').read()
    jd_cl=clean_text(jd_txt,False)
    jd_sent=sent_tokenize(jd_cl)
    res_txt=open('Z:\\SEMP_RES1.txt').read()
    res_cl=clean_text(res_txt,False)
    res_sent=sent_tokenize(res_cl)

    while '.' in jd_sent: jd_sent.remove('.')
    while '.' in res_sent: res_sent.remove('.')
    return(jd_cl,jd_sent,res_cl,res_sent)

### Here it starts

In [128]:
##################################################################################
# Accepts a list of text strings ( document(s) and returns info about Topics and writes JSON File)

def get_topics():
    from bertopic import BERTopic
    from sklearn.datasets import fetch_20newsgroups
    import json

    JPATH='Z:/FILES/OUTPUT/JSON/'
    #docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

    # I used our jd and resume from semperit and Jon
    jd_cl,jd_sent,res_cl,res_sent=load_files()

    #we use sentnteces in JD Document
    docs=jd_sent

    #instantiate a model
    topic_model = BERTopic(embedding_model='sentence-transformers/paraphrase-MiniLM-L6-v2',min_topic_size=2,calculate_probabilities=True)

    #fit your documents ( List[str]) to the model
    # we will get different results each time we run - reason:use of stochatic models
    topics, _ = topic_model.fit_transform(docs)
    info_df=topic_model.get_topic_info()

    t_scores=topic_model.get_topics()

    # you even can use graphics
    #topic_model.visualize_topics()

    #write to jason file 
    result = info_df.to_json(orient="split")
    print(result)
    with open(JPATH+'BERTopic.json', 'w') as f:
        f.write(result)
        f.close
    return(topics,result,info_df,t_scores)

In [129]:
topics,results,info_df,t_scores=get_topics()
info_df


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{"columns":["Topic","Count","Name"],"index":[0,1,2,3],"data":[[0,14,"0_location_vienna_work_experience"],[1,9,"1_management_service_strategy_maintain"],[-1,5,"-1_service_regulations_ensures_guaranteed"],[2,3,"2_desk_service_complaints_efficiency"]]}


,Topic,Count,Name
0,0,14,0_location_vienna_work_experience
1,1,9,1_management_service_strategy_maintain
2,-1,5,-1_service_regulations_ensures_guaranteed
3,2,3,2_desk_service_complaints_efficiency
